In [79]:
import functools
import math

In [80]:
def removeLetters(boxes):
    rst = []
    n = int(math.pow(len(boxes),1/2))
    for i,box in enumerate(boxes):
      a = int(box[0][1]) # Solamente existe Z9 2 valores, A,B,C..Z y 1,2...9
      b = box[1]
      if a!=1 and len(boxes)>2:
        b = b.replace('B','')
      if (i+1)>len(boxes)-n:
        for lt in b:
            rst.append(f'{boxes[i][0]} {lt}')
        continue
      b = b.replace('R','')
      for lt in b:
            rst.append(f'{boxes[i][0]} {lt}')
    newRst = {}
    for v in rst:
        a = v.split(' ')[0]
        b = v.split(' ')[1]
        if newRst.get(a,0)!=0:
            newRst[a] = newRst[a]+""+b
        else:
            newRst[a] = b
    rst = []
    for key, value in newRst.items():
        temp = (key,value)
        rst.append(temp)
    return rst

def flatten_boxes(boxes):

    return functools.reduce(
        lambda m, box: m + list(map(lambda side: f'{box[0]} {side}',list(box[1])))
    , boxes, [])
#removeLetters([('A1', 'LTRB'), ('A2', 'LTRB'), ('B1', 'LTRB'), ('B2', 'LTRB')])

In [81]:
def remove_move(boxes, move):
    move_box, move_side = move.split(' ')
    clone = boxes[:]

    for i, box in enumerate(clone):
        if box[0] == move_box:
            new_sides = box[1].replace(move_side, '')
            clone[i] = (box[0], new_sides) if new_sides != '' else None
            break

    nonone = list(filter(lambda b: not (b is None), clone))

    return nonone

In [82]:
def is_candidate_closed_by(box, move):
    move_box, move_side = move.split(' ')

    opposites = {
        'T': 'B',
        'B': 'T',
        'L': 'R',
        'R': 'L'
    }

    if box[0] == move_box and box[1] == move_side: # closes box
        return True
    if box[0] != move_box and box[1] == opposites[move_side] and move_box[0] == box[0][0]: # closes same cloumn
        return True
    elif box[0] != move_box and box[1] == opposites[move_side] and move_box[1] == box[0][1]: # closes same row
        return True

    return False

In [83]:
def closed_boxes(boxes, move):
    candidates = list(filter(lambda b: len(b[1]) == 1, boxes))
    closed = list(filter(lambda b: is_candidate_closed_by(b, move), candidates))
    
    return len(closed)

In [84]:
# READY: primera optimizacion, eliminar jugadas repetidas
# Se hace en el metodo removeLetters, cuyo resultado se pasa como entrada al primer minimax que se hace
def minimax(boxes, max_turn, max_score, min_score,lvl,alpha,beta):
    if lvl == 3: # READY: segunda optimizacion, cortar en profundidad y evaluar
        return max_score - min_score
    if len(boxes) == 0: 
        return max_score - min_score

    children =  flatten_boxes(boxes)
    #boxes = removeLettersTuple(boxes)
    if max_turn:
        max_eval = -1000
        good_move=''
        for move in children:
            new_moves = remove_move(boxes, move)
            closed_b = closed_boxes(boxes, move)
            if closed_b>0:
                new_moves = reArrangeBoxes(new_moves,move)
            new_max_score = max_score + closed_b # tercera optimizacion, usar numero de cajas cerradas para ordenar movimientos
            rating = minimax(new_moves, not max_turn, new_max_score, min_score,lvl+1,alpha,beta)
            if(rating>max_eval):
                max_eval=rating
                good_move=move
            alpha = max(alpha, max_eval) # READY: cuarta optimizacion, usar poda alfa-beta
            if alpha >= beta:
                break
        if(lvl==0):
            print(good_move)
        return max_eval
    else:
        min_eval = 1000
        for move in children:
            new_moves = remove_move(boxes, move)
            closed_b = closed_boxes(boxes, move)
            if closed_b>0:
                new_moves = reArrangeBoxes(new_moves,move)
            new_min_score = min_score + closed_b
            rating = minimax(new_moves, not max_turn, max_score, new_min_score,lvl+1,alpha,beta)
            min_eval = min(rating, min_eval)
            beta = min(beta, min_eval) # READY: cuarta optimizacion, usar poda alfa-beta
            if beta <= alpha:
                break
        return min_eval


In [102]:
def reArrangeBoxes(moves, move):
    return sorted(moves, key=lambda x: len(x[1]), reverse=True)
    

In [138]:
import datetime

start = datetime.datetime.now()
#('A1', 'LTBR'), ('A2', 'LTBR'), ('A3', 'LTBR'), ('B1', 'LTRB'), ('B2', 'LTRB'), ('B3', 'LTBR')
v = minimax(removeLetters([('A1', 'LTBR'), ('A2', 'LTBR'), ('B1', 'LTRB'), ('B2', 'LTRB')]), True, 0, 0,0,-1000, 1000)

end = datetime.datetime.now()

print((end-start).total_seconds()*1,v)

A1 L
0.000999 0
